In [2]:
import numpy as np
import random

In [3]:
def look_for_targets(free_space, start, targets, logger=None):
    """
    Find direction of closest target that can be reached via free tiles.

    Performs a breadth-first search of the reachable free tiles until a target is encountered.
    If no target can be reached, the path that takes the agent closest to any target is chosen.

    Args:
        free_space: Boolean numpy array. True for free tiles and False for obstacles.
        start: the coordinate from which to begin the search.
        targets: list or array holding the coordinates of all target tiles.
        logger: optional logger object for debugging.
    Returns:
        coordinate of first step towards closest target or towards tile closest to any target.
    """
    
    
    if len(targets) == 0: return None

    frontier    = [start]         # tree leaves
    parent_dict = {start: start}  # branching points
    dist_so_far = {start: 0}      # branch lengths
    best_ones   = []
    best_dist   = np.sum(np.abs(np.subtract(targets, start)), axis=1).min()
    found_one   = False

    while len(frontier) > 0:   # While there still are reachable tiles
        current = frontier.pop(0)
        
        # Find distance from current position to all targets, track closest
        d = np.sum(np.abs(np.subtract(targets, current)), axis=1).min()
        current_dist = d + dist_so_far[current]
        if d == 0:   # In case there is a reachable coin, stop only if you have found a path to it.
            # Found path to a target's exact position, mission accomplished!
            best_ones.append(current)
            best_dist = current_dist
            found_one = True
        elif current_dist == best_dist:   # In case no coin is reachable, find reachable tile closest to closest coin.
            best_ones.append(current)   
        elif current_dist < best_dist:
            best_ones = [current]
            best_dist = current_dist
        
        if found_one and dist_so_far[current] >= best_dist:   # If one target has already been found and this tile doesn't have a target, forget about it.
            # Forget about current tile
            continue    
        else:   # else expand the frontier by adding neighbors        
            # Add unexplored free neighboring tiles to the queue in a random order
            x, y       = current
            directions = [(x, y - 1), (x + 1, y), (x, y + 1), (x - 1, y)]   # UP, RIGHT, DOWN, LEFT from (x, y)
            neighbors  = [(x_dir, y_dir)  for (x_dir, y_dir) in directions  if free_space[x_dir, y_dir]]
            random.shuffle(neighbors)
            for neighbor in neighbors:
                if neighbor not in parent_dict:
                    frontier.append(neighbor)
                    parent_dict[neighbor] = current
                    dist_so_far[neighbor] = dist_so_far[current] + 1
    
    
    if logger: logger.debug(f'Suitable target(s) found at {best_ones}')
    
    # Determine the first step (best direction(s)) towards the best found target tile(s)
    directions = []
    while len(best_ones) > 0:
        current = best_ones.pop(0)
        parent = parent_dict[current]
        if parent == start:
            directions.append(current)
        elif parent not in best_ones:
            best_ones.append(parent)
    
    return directions

In [ ]:
diction = {"a": "b", "b": "c"}